In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../data'))
import data_preparation_sm as sm
import modeling as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('../data/transformed_data1.csv')

C:\Users\Hp\AppData\Local\Temp\ipykernel_1976\3167079008.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/transformed_data1.csv')


In [3]:
# Select a subset of the data for quicker processing
df = df.sample(n=100, random_state=42) 

In [4]:
# prepare data
df = sm.feature_engineering(df)

c:\Users\Hp\Desktop\Kifiya\Week3\ACIS-Claim-Analytics\scripts\data_preparation_sm.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], errors='coerce')


In [5]:
df = sm.encode_categorical_data(df)

In [6]:
df = sm.scale_numerical_features(df)

In [7]:
# Set the option to display all columns
pd.set_option('display.max_columns', None)
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,CustomValueEstimate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,CrossBorder,SumInsured,CalculatedPremiumPerTerm,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,Year,Month,AgeOfVehicle,VehicleValue,DriverExperience,Region_VehicleType_Free State_Passenger Vehicle,Region_VehicleType_Gauteng_Heavy Commercial,Region_VehicleType_Gauteng_Medium Commercial,Region_VehicleType_Gauteng_Passenger Vehicle,Region_VehicleType_KwaZulu-Natal_Heavy Commercial,Region_VehicleType_KwaZulu-Natal_Light Commercial,Region_VehicleType_KwaZulu-Natal_Medium Commercial,Region_VehicleType_KwaZulu-Natal_Passenger Vehicle,Region_VehicleType_Limpopo_Medium Commercial,Region_VehicleType_Limpopo_Passenger Vehicle,Region_VehicleType_Mpumalanga_Medium Commercial,Region_VehicleType_Mpumalanga_Passenger Vehicle,Region_VehicleType_North West_Light Commercial,Region_VehicleType_North West_Medium Commercial,Region_VehicleType_North West_Passenger Vehicle,Region_VehicleType_Western Cape_Heavy Commercial,Region_VehicleType_Western Cape_Medium Commercial,Region_VehicleType_Western Cape_Passenger Vehicle,VehicleMake_CoverType_1,VehicleMake_CoverType_2,VehicleMake_CoverType_3,VehicleMake_CoverType_4,VehicleMake_CoverType_5,VehicleMake_CoverType_6,VehicleMake_CoverType_7,VehicleMake_CoverType_8,VehicleMake_CoverType_9,VehicleMake_CoverType_10,VehicleMake_CoverType_11,VehicleMake_CoverType_12,VehicleMake_CoverType_13,VehicleMake_CoverType_14,VehicleMake_CoverType_15,VehicleMake_CoverType_16,VehicleMake_CoverType_17,VehicleMake_CoverType_18,VehicleMake_CoverType_19,VehicleMake_CoverType_20,VehicleMake_CoverType_21,VehicleMake_CoverType_22,VehicleMake_CoverType_23,VehicleMake_CoverType_24,VehicleMake_CoverType_25,VehicleMake_CoverType_26,VehicleMake_CoverType_27,VehicleMake_CoverType_28,VehicleMake_CoverType_29,VehicleMake_CoverType_30,VehicleMake_CoverType_31,VehicleMake_CoverType_32,VehicleMake_CoverType_33,VehicleMake_CoverType_34,VehicleMake_CoverType_35,VehicleMake_CoverType_36,VehicleMake_CoverType_37,VehicleMake_CoverType_38,VehicleMake_CoverType_39,VehicleMake_CoverType_40,VehicleMake_CoverType_41,VehicleMake_CoverType_42,VehicleMake_CoverType_43,VehicleMake_CoverType_44,VehicleMake_CoverType_45,VehicleMake_CoverType_46,VehicleMake_CoverType_47,VehicleMake_CoverType_48,VehicleMake_CoverType_49,VehicleMake_CoverType_50,VehicleMake_CoverType_51,VehicleMake_CoverType_52,VehicleMake_CoverType_53,VehicleMake_CoverType_54,VehicleMake_CoverType_55,VehicleMake_CoverType_56,VehicleMake_CoverType_57,VehicleMake_CoverType_58,VehicleMake_CoverType_59,VehicleMake_CoverType_60,VehicleMake_CoverType_61,VehicleMake_CoverType_62,VehicleMake_CoverType_63,VehicleMake_CoverType_64,VehicleMake_CoverType_65,VehicleMake_CoverType_66,VehicleMake_CoverType_67,VehicleMake_CoverType_68,VehicleMake_CoverType_69,VehicleMake_CoverType_70,VehicleMake_CoverType_71,VehicleMake_CoverType_72,VehicleMake_CoverType_73,VehicleMake_CoverType_74,CoverGroup_Premium_1,CoverGroup_Premium_2,CoverGroup_Premium_3,CoverGroup_Premium_4,CoverGroup_Premium_5,CoverGroup_Premium_6,CoverGroup_Premium_7,CoverGroup_Premium_8,CoverGroup_Premium_9,CoverGroup_Premium_10,CoverGroup_Premium_11,CoverGroup_Premium_12,CoverGroup_Premium_13,CoverGroup_Premium_14,CoverGroup_Premium_15,CoverGroup_Premium_16,CoverGroup_Premium_17,CoverGroup_Premium_18,CoverGroup_Premium_19,CoverGroup_Premium_20,CoverGroup_Premium_21,CoverGroup_Premium_22,CoverGroup_Premium_23,CoverGroup_Premium_24,CoverGroup_Premium_25,CoverGroup_Premium_26,CoverGroup_Premium_27,CoverGroup_Premium_28,CoverGroup_Premium_29,CoverGroup_Premium_30,CoverGroup_Premi

In [8]:
# Split data into training and testing sets
target_columns = ['TotalPremium', 'TotalClaims']
X_train, X_test, y_train, y_test = train_test_split(df.drop(target_columns, axis=1), 
                                                    df[target_columns], test_size=0.3, random_state=42)

In [9]:
# Feature selection
X_train_selected = m.feature_selection_dask(X_train, y_train['TotalPremium'])

In [24]:
# Handle imbalanced data
X_train_upsampled, y_train_upsampled = m.handle_imbalanced_data_regression(X_train_selected, y_train)

In [42]:
# Train and evaluate models
m.train_and_evaluate_models(X_train_upsampled, X_test, y_train_upsampled, y_test, ['TotalPremium'])

In [9]:
print(X_train.dtypes)

UnderwrittenCoverID                                                   float64
PolicyID                                                              float64
TransactionMonth                                                       object
IsVATRegistered                                                          bool
Citizenship                                                            object
                                                                 ...         
CoverGroup_Claims_Motor Comprehensive_37094.9736842105    Sparse[bool, False]
CoverGroup_Claims_Motor Comprehensive_8119.9473684211     Sparse[bool, False]
CoverGroup_Claims_Standalone passenger liability_0.0      Sparse[bool, False]
CoverGroup_Claims_Third Party Only_0.0                    Sparse[bool, False]
CoverGroup_Claims_Trailer_0.0                             Sparse[bool, False]
Length: 21937, dtype: object


In [43]:
# Interpret the best model
best_model = RandomForestRegressor(n_estimators=100) 
m.interpret_model(best_model, X_test)

AssertionError: Model has no `estimators_`! Have you called `model.fit`?